Hi Andriy!

This is the code i've been using as of now for the news search

In [19]:
import os
from datetime import date
from dotenv import load_dotenv

from langchain import hub
from langchain.tools import Tool
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain.prompts import ChatPromptTemplate

from langchain_aws import ChatBedrock
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import AIMessage, HumanMessage

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.wikidata.tool import WikidataAPIWrapper, WikidataQueryRun
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsInput

from mediawikiapi import MediaWikiAPI
import boto3



# Create the AWS client for the Bedrock runtime with boto3
aws_client = boto3.client(service_name="bedrock-runtime")

In [20]:
load_dotenv(dotenv_path=r"C:\Users\Evan\Desktop\GSB 570\env_for_db", override=True)
aws_access_key = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
region_name = os.getenv("AWS_DEFAULT_REGION")
output_format = os.getenv("AWS_DEFAULT_OUTPUT")


In [21]:

# 1. Load API key
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
tavily = TavilySearchResults(tavily_api_key=TAVILY_API_KEY, k=3)

# 2. Function to perform expanded search with dynamic date
def expanded_tavily_search(user_query: str) -> str:
    today = date.today().strftime("%B %d, %Y")  # e.g., "May 24, 2025"

    queries = [
        f"{user_query} {today}",
        f"U.S. car inventory levels {today}",
        f"U.S. vehicle pricing news {today}",
        f"automotive supply chain issues {today}",
        f"EV sales trends {today}",
        f"auto dealer news {today}"
    ]

    results = []
    for q in queries:
        res = tavily.run(q)
        if res:
            results.extend(res)

    # Deduplicate by URL
    seen = set()
    unique = []
    for r in results:
        if r["url"] not in seen:
            unique.append(r)
            seen.add(r["url"])

    if not unique:
        return f"No relevant updates found for {today}."

    return "\n\n".join(
        f"{r['title']}\n{r['url']}\n{r.get('content', '')[:300]}..." for r in unique[:5]
    )

# 3. Wrap it as a LangChain tool
expanded_tavily_tool = Tool.from_function(
    name="Broader Tavily Search",
    description="Searches news about the U.S. car market from multiple angles using Tavily.",
    func=expanded_tavily_search
)


In [22]:
# 1. Load Tavily API Key from environment
TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')
if not TAVILY_API_KEY:
    raise ValueError("Missing TAVILY_API_KEY from environment")

# 2. Set up Tavily tool

tools = [expanded_tavily_tool]

# 3. Set up query
today = date.today().strftime("%B %d, %Y")
query = (
    f"Search for and summarize the most relevant U.S. car market news as of {today}. "
    f"Focus only on information published today or in the last 48 hours — do not include general 2025 forecasts or outdated articles. "
    f"Highlight developments that affect dealership operations, such as vehicle pricing changes, inventory levels, sales trends, and logistics or supply chain issues. "
    f"The summary should be concise, factual, and directly useful for car dealership managers. "
    f"If there are no credible updates published today or in the last 48 hours, respond with: 'No relevant updates found for {today}.'"
)

# 4. Configure LLM (Claude via Bedrock)
llm = ChatBedrock(
    client=aws_client,
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs={
        "max_tokens": 2048,
        "temperature": 0.0,
        "top_k": 250,
        "top_p": 0.9,
        "stop_sequences": ["\n\nHuman"]
    }
)

# 5. Load structured prompt
prompt = hub.pull("hwchase17/structured-chat-agent")

# 6. Create agent
agent = create_structured_chat_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

# 7. Invoke
response = agent_executor.invoke({"input": query})
print(response)


c:\Users\Evan\Desktop\GSB 570\gsb570env\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "Broader Tavily Search",
  "action_input": "U.S. car market news May 24 2025"
}
```
Car dealers have a bigger problem than tariffs - TheStreet
https://www.thestreet.com/automotive/car-dealers-have-a-bigger-problem-than-tariffs
This problem could reduce long-term demand for new vehicles. Author: Christy Rakoczy. May 24, 2025 7:37 AM EDT....

USA - Automotive Sales volume, 2025 - MarkLines
https://www.marklines.com/en/statistics/flash_sales/automotive-sales-in-usa-by-month
By vehicle type, sales of light trucks were up 8.9% y-o-y to 919,290 units, while passenger cars decreased 15.2% to 193,654 units. [...] | :---: | --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | Ford | Ford F-Series | 55,262 | 51,829 | 6.6% | 107,717 | 100,531 | 7.1% |
| 2 | Chevrolet | Silver...

New Cars with the Most Available Inventory - RealCarTips.com
https://www.realcartips.com/news/1439-cars-with-most-inventory.shtml
The vehicle with th

In [23]:
print(response['output'])


Based on the search results from the last 48 hours, here are the most relevant updates for U.S. car dealerships as of May 24, 2025:

- New vehicle inventory levels remain tight, with the Ford F-150 having the most units (52,559) available nationwide. However, overall light truck sales were up 8.9% year-over-year while passenger car sales declined 15.2%.

- For used cars, prices are rising again after months of declines. The average used listing price is now $25,128 as the spring buying season begins and tariff concerns drive demand before potential further price hikes. 

- The fastest selling vehicles based on market day supply are the Lexus GX (24 days), Cadillac Escalade (24 days), and Lexus RX (28 days). The slowest sellers include the BMW X6 and Audi Q7.

- A potential long-term threat to new vehicle demand is highlighted, though details are not provided. Dealerships should monitor this developing issue.

No major pricing adjustments, supply chain disruptions, or other operational 